# Factorizations and other fun
Based on work by Andreas Noack Jensen (MIT) (http://www.econ.ku.dk/phdstudent/noack/)

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.6170120247910962
 1.6644545921552723
 0.5894827313316517

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.124045  1.0       0.0
 0.127652  0.281901  1.0
U factor:
3×3 Matrix{Float64}:
 0.302203  0.42203    0.940222
 0.0       0.694737   0.715808
 0.0       0.0       -0.0206226

In [3]:
?lu

search: lu lu! LU Alu flush values include include_string include_dependency



```
lu(A, pivot=Val(true); check = true) -> F::LU
```

Compute the LU factorization of `A`.

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

In most cases, if `A` is a subtype `S` of `AbstractMatrix{T}` with an element type `T` supporting `+`, `-`, `*` and `/`, the return type is `LU{T,S{T}}`. If pivoting is chosen (default) the element type should also support [`abs`](@ref) and [`<`](@ref).

The individual components of the factorization `F` can be accessed via [`getproperty`](@ref):

| Component | Description                         |
|:--------- |:----------------------------------- |
| `F.L`     | `L` (lower triangular) part of `LU` |
| `F.U`     | `U` (upper triangular) part of `LU` |
| `F.p`     | (right) permutation `Vector`        |
| `F.P`     | (right) permutation `Matrix`        |

Iterating the factorization produces the components `F.L`, `F.U`, and `F.p`.

The relationship between `F` and `A` is

`F.L*F.U == A[F.p, :]`

`F` further supports the following functions:

| Supported function  | `LU` | `LU{T,Tridiagonal{T}}` |
|:------------------- |:---- |:---------------------- |
| [`/`](@ref)         | ✓    |                        |
| [`\`](@ref)         | ✓    | ✓                      |
| [`inv`](@ref)       | ✓    | ✓                      |
| [`det`](@ref)       | ✓    | ✓                      |
| [`logdet`](@ref)    | ✓    | ✓                      |
| [`logabsdet`](@ref) | ✓    | ✓                      |
| [`size`](@ref)      | ✓    | ✓                      |

# Examples

```jldoctest
julia> A = [4 3; 6 3]
2×2 Matrix{Int64}:
 4  3
 6  3

julia> F = lu(A)
LU{Float64, Matrix{Float64}}
L factor:
2×2 Matrix{Float64}:
 1.0       0.0
 0.666667  1.0
U factor:
2×2 Matrix{Float64}:
 6.0  3.0
 0.0  1.0

julia> F.L * F.U == A[F.p, :]
true

julia> l, u, p = lu(A); # destructuring via iteration

julia> l == F.L && u == F.U && p == F.p
true
```

---

```
lu(A::SparseMatrixCSC; check = true) -> F::UmfpackLU
```

Compute the LU factorization of a sparse matrix `A`.

For sparse `A` with real or complex element type, the return type of `F` is `UmfpackLU{Tv, Ti}`, with `Tv` = [`Float64`](@ref) or `ComplexF64` respectively and `Ti` is an integer type ([`Int32`](@ref) or [`Int64`](@ref)).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

The individual components of the factorization `F` can be accessed by indexing:

| Component | Description                         |
|:--------- |:----------------------------------- |
| `L`       | `L` (lower triangular) part of `LU` |
| `U`       | `U` (upper triangular) part of `LU` |
| `p`       | right permutation `Vector`          |
| `q`       | left permutation `Vector`           |
| `Rs`      | `Vector` of scaling factors         |
| `:`       | `(L,U,p,q,Rs)` components           |

The relation between `F` and `A` is

`F.L*F.U == (F.Rs .* A)[F.p, F.q]`

`F` further supports the following functions:

  * [`\`](@ref)
  * [`cond`](@ref)
  * [`det`](@ref)

!!! note
    `lu(A::SparseMatrixCSC)` uses the UMFPACK library that is part of SuiteSparse. As this library only supports sparse matrices with [`Float64`](@ref) or `ComplexF64` elements, `lu` converts `A` into a copy that is of type `SparseMatrixCSC{Float64}` or `SparseMatrixCSC{ComplexF64}` as appropriate.



In [4]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [5]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0

In [6]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.124045  1.0       0.0
 0.127652  0.281901  1.0

In [7]:
Alu.U

3×3 Matrix{Float64}:
 0.302203  0.42203    0.940222
 0.0       0.694737   0.715808
 0.0       0.0       -0.0206226

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [8]:
A\b

3-element Vector{Float64}:
 1.0
 1.0
 1.0

In [9]:
Alu\b

3-element Vector{Float64}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [10]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [11]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.122125   0.955012  -0.270253
 -0.984526  -0.151044  -0.0888543
 -0.125677   0.25522    0.95868
R factor:
3×3 Matrix{Float64}:
 -0.306952  -0.538122  -1.06519
  0.0        0.713466   0.729843
  0.0        0.0       -0.0197705

In [12]:
?qr

search: qr qr! QR QRPivoted Aqr sqrt isqrt QuickSort PartialQuickSort



```
qr(A, pivot=Val(false); blocksize) -> F
```

Compute the QR factorization of the matrix `A`: an orthogonal (or unitary if `A` is complex-valued) matrix `Q`, and an upper triangular matrix `R` such that

$$
A = Q R
$$

The returned object `F` stores the factorization in a packed format:

  * if `pivot == Val(true)` then `F` is a [`QRPivoted`](@ref) object,
  * otherwise if the element type of `A` is a BLAS type ([`Float32`](@ref), [`Float64`](@ref), `ComplexF32` or `ComplexF64`), then `F` is a [`QRCompactWY`](@ref) object,
  * otherwise `F` is a [`QR`](@ref) object.

The individual components of the decomposition `F` can be retrieved via property accessors:

  * `F.Q`: the orthogonal/unitary matrix `Q`
  * `F.R`: the upper triangular matrix `R`
  * `F.p`: the permutation vector of the pivot ([`QRPivoted`](@ref) only)
  * `F.P`: the permutation matrix of the pivot ([`QRPivoted`](@ref) only)

Iterating the decomposition produces the components `Q`, `R`, and if extant `p`.

The following functions are available for the `QR` objects: [`inv`](@ref), [`size`](@ref), and [`\`](@ref). When `A` is rectangular, `\` will return a least squares solution and if the solution is not unique, the one with smallest norm is returned. When `A` is not full rank, factorization with (column) pivoting is required to obtain a minimum norm solution.

Multiplication with respect to either full/square or non-full/square `Q` is allowed, i.e. both `F.Q*F.R` and `F.Q*A` are supported. A `Q` matrix can be converted into a regular matrix with [`Matrix`](@ref).  This operation returns the "thin" Q factor, i.e., if `A` is `m`×`n` with `m>=n`, then `Matrix(F.Q)` yields an `m`×`n` matrix with orthonormal columns.  To retrieve the "full" Q factor, an `m`×`m` orthogonal matrix, use `F.Q*Matrix(I,m,m)`.  If `m<=n`, then `Matrix(F.Q)` yields an `m`×`m` orthogonal matrix.

The block size for QR decomposition can be specified by keyword argument `blocksize :: Integer` when `pivot == Val(false)` and `A isa StridedMatrix{<:BlasFloat}`. It is ignored when `blocksize > minimum(size(A))`.  See [`QRCompactWY`](@ref).

!!! compat "Julia 1.4"
    The `blocksize` keyword argument requires Julia 1.4 or later.


# Examples

```jldoctest
julia> A = [3.0 -6.0; 4.0 -8.0; 0.0 1.0]
3×2 Matrix{Float64}:
 3.0  -6.0
 4.0  -8.0
 0.0   1.0

julia> F = qr(A)
LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.6   0.0   0.8
 -0.8   0.0  -0.6
  0.0  -1.0   0.0
R factor:
2×2 Matrix{Float64}:
 -5.0  10.0
  0.0  -1.0

julia> F.Q * F.R == A
true
```

!!! note
    `qr` returns multiple types because LAPACK uses several representations that minimize the memory storage requirements of products of Householder elementary reflectors, so that the `Q` and `R` matrices can be stored compactly rather as two separate dense matrices.


---

```
qr(A::SparseMatrixCSC; tol=_default_tol(A), ordering=ORDERING_DEFAULT) -> QRSparse
```

Compute the `QR` factorization of a sparse matrix `A`. Fill-reducing row and column permutations are used such that `F.R = F.Q'*A[F.prow,F.pcol]`. The main application of this type is to solve least squares or underdetermined problems with [`\`](@ref). The function calls the C library SPQR.

!!! note
    `qr(A::SparseMatrixCSC)` uses the SPQR library that is part of SuiteSparse. As this library only supports sparse matrices with [`Float64`](@ref) or `ComplexF64` elements, as of Julia v1.4 `qr` converts `A` into a copy that is of type `SparseMatrixCSC{Float64}` or `SparseMatrixCSC{ComplexF64}` as appropriate.


# Examples

```jldoctest
julia> A = sparse([1,2,3,4], [1,1,2,2], [1.0,1.0,1.0,1.0])
4×2 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
 1.0   ⋅
 1.0   ⋅
  ⋅   1.0
  ⋅   1.0

julia> qr(A)
SuiteSparse.SPQR.QRSparse{Float64, Int64}
Q factor:
4×4 SuiteSparse.SPQR.QRSparseQ{Float64, Int64}:
 -0.707107   0.0        0.0       -0.707107
  0.0       -0.707107  -0.707107   0.0
  0.0       -0.707107   0.707107   0.0
 -0.707107   0.0        0.0        0.707107
R factor:
2×2 SparseMatrixCSC{Float64, Int64} with 2 stored entries:
 -1.41421    ⋅
   ⋅       -1.41421
Row permutation:
4-element Vector{Int64}:
 1
 3
 4
 2
Column permutation:
2-element Vector{Int64}:
 1
 2
```


Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [13]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.122125   0.955012  -0.270253
 -0.984526  -0.151044  -0.0888543
 -0.125677   0.25522    0.95868

In [14]:
Aqr.R

3×3 Matrix{Float64}:
 -0.306952  -0.538122  -1.06519
  0.0        0.713466   0.729843
  0.0        0.0       -0.0197705

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [15]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.6616442806377435
 -0.4595332606156878
  2.6425825943260914
vectors:
3×3 Matrix{Float64}:
  0.854193  -0.224749  -0.468873
 -0.508131  -0.552044  -0.661097
 -0.110258   0.802954  -0.585755

The values and the vectors can be extracted from the Eigen type by special indexing

In [16]:
AsymEig.values

3-element Vector{Float64}:
 -0.6616442806377435
 -0.4595332606156878
  2.6425825943260914

In [17]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.854193  -0.224749  -0.468873
 -0.508131  -0.552044  -0.661097
 -0.110258   0.802954  -0.585755

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [18]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0          9.99201e-16  9.4369e-16
  6.66134e-16  1.0          1.22125e-15
 -2.22045e-16  4.44089e-16  1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [19]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [20]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [21]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-1.7806400370270028

In [22]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [23]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [24]:
@time eigvals(Asym);

  0.460447 seconds (133.94 k allocations: 15.454 MiB, 3.51% gc time, 36.15% compilation time)


In [25]:
@time eigvals(Asym_noisy);

  1.064262 seconds (13 allocations: 7.920 MiB)


In [26]:
@time eigvals(Asym_explicit);

  0.293277 seconds (5.88 k allocations: 8.358 MiB, 3.56% compilation time)


In [27]:
using BenchmarkTools  


In [28]:
@benchmark eigvals($Asym)

BenchmarkTools.Trial: 
  memory estimate:  7.99 MiB
  allocs estimate:  11
  --------------
  minimum time:     265.792 ms (0.00% GC)
  median time:      275.191 ms (0.00% GC)
  mean time:        274.591 ms (0.31% GC)
  maximum time:     283.508 ms (0.00% GC)
  --------------
  samples:          19
  evals/sample:     1

In [29]:
@benchmark eigvals($Asym_noisy)

BenchmarkTools.Trial: 
  memory estimate:  7.92 MiB
  allocs estimate:  13
  --------------
  minimum time:     1.049 s (0.00% GC)
  median time:      1.056 s (0.00% GC)
  mean time:        1.068 s (0.00% GC)
  maximum time:     1.112 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

In [30]:
@benchmark eigvals($Asym_explicit)

BenchmarkTools.Trial: 
  memory estimate:  7.99 MiB
  allocs estimate:  11
  --------------
  minimum time:     264.836 ms (0.00% GC)
  median time:      271.155 ms (0.00% GC)
  mean time:        271.757 ms (0.26% GC)
  maximum time:     278.987 ms (0.00% GC)
  --------------
  samples:          19
  evals/sample:     1

In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [31]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.819444 seconds (782.01 k allocations: 226.927 MiB, 2.45% gc time, 28.81% compilation time)


6.19877544505124

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [32]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [33]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 2//5  1//1  2//5
 1//1  1//1  7//10
 2//5  4//5  2//5

In [34]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  9//5
 27//10
  8//5

In [35]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [36]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  0//1  0//1
 2//5  1//1  0//1
 2//5  2//3  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  1//1  7//10
 0//1  3//5  3//25
 0//1  0//1  1//25

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [38]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

5×5 Matrix{Int64}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514494
  542.4677301466137
vectors:
5×5 Matrix{Float64}:
 -0.147575  -0.647178  -0.010882    0.548903   -0.507907
 -0.256795   0.173068  -0.834628   -0.239864   -0.387253
 -0.185537  -0.239762   0.422161   -0.731925   -0.440631
  0.819704   0.247506   0.0273194   0.0366447  -0.514526
 -0.453805   0.657619   0.352577    0.322668   -0.364928

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514494
  542.4677301466137

In [43]:
@assert A_eigv ≈ [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [48]:
@assert isapprox(A_diag, [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468], atol = 0.001)

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [50]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]